<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](./data/mountain/art1131.jpg)|![](./data/forest/cdmc317.jpg)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

from PIL import Image
from sklearn.model_selection import train_test_split

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential, Model # This is the functional API

In [0]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [0]:
!rm -r sample_data/

In [0]:
mkdir forest && mv *.jpg forest

In [0]:
mkdir mountain && mv *.jpg mountain

In [0]:
forest = ['./forest/'+ image for image in os.listdir('./forest')]
mountain = ['./mountain/'+ image for image in os.listdir('./mountain')]

In [0]:
y0 = np.zeros(len(forest))
y1 = np.ones(len(mountain))
y = np.hstack((y0,y1))

In [0]:
# Create data arrays (X)
X = []
for image in (forest + mountain):
  image_array = np.asarray(Image.open(image))
  X.append(image_array)

X = np.array(X) / 255 # normalize pixel values by dividing by 255

In [12]:
X.shape, y.shape

((702, 256, 256, 3), (702,))

## Instatiate Model

In [13]:
resnet = ResNet50(weights='imagenet', include_top=False)

for layer in resnet.layers:
    layer.trainable = False

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
94773248/94765736 [==============================] - 1s 0us/step


In [0]:
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

In [15]:
model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


## Fit Model

In [16]:
resnet = model.fit(X, y, validation_split=0.15, epochs=10)

Train on 596 samples, validate on 106 samples
Epoch 1/10
596/596 [==============================] - 213s 357ms/sample - loss: 0.7574 - acc: 0.8171 - val_loss: 2.5083 - val_acc: 0.0000e+00
Epoch 2/10
596/596 [==============================] - 210s 353ms/sample - loss: 0.0441 - acc: 0.9815 - val_loss: 2.4353 - val_acc: 0.0000e+00
Epoch 3/10
596/596 [==============================] - 209s 351ms/sample - loss: 0.0491 - acc: 0.9815 - val_loss: 2.9637 - val_acc: 0.0000e+00
Epoch 4/10
596/596 [==============================] - 209s 351ms/sample - loss: 0.0535 - acc: 0.9782 - val_loss: 2.1558 - val_acc: 0.0000e+00
Epoch 5/10
596/596 [==============================] - 211s 353ms/sample - loss: 0.0356 - acc: 0.9815 - val_loss: 1.7897 - val_acc: 0.0000e+00
Epoch 6/10
596/596 [==============================] - 209s 350ms/sample - loss: 0.0102 - acc: 0.9983 - val_loss: 1.5227 - val_acc: 0.0000e+00
Epoch 7/10
596/596 [==============================] - 209s 351ms/sample - loss: 0.0072 - acc: 0.9983 -

# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [17]:
# set up architecture:
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 230400)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                1

In [0]:
# Compile Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
# Fit Model
customCNN = model.fit(X, y, epochs=10, validation_split=0.15)

Train on 596 samples, validate on 106 samples
Epoch 1/10
596/596 [==============================] - 65s 108ms/sample - loss: 1.2632 - acc: 0.7282 - val_loss: 0.4796 - val_acc: 0.8019
Epoch 2/10
596/596 [==============================] - 64s 107ms/sample - loss: 0.3960 - acc: 0.8406 - val_loss: 0.2743 - val_acc: 0.9528
Epoch 3/10
596/596 [==============================] - 64s 107ms/sample - loss: 0.1799 - acc: 0.9346 - val_loss: 0.1868 - val_acc: 0.9245
Epoch 4/10
596/596 [==============================] - 64s 107ms/sample - loss: 0.1319 - acc: 0.9362 - val_loss: 0.2596 - val_acc: 0.9057
Epoch 5/10
596/596 [==============================] - 63s 106ms/sample - loss: 0.0982 - acc: 0.9681 - val_loss: 0.1562 - val_acc: 0.9434
Epoch 6/10
596/596 [==============================] - 63s 106ms/sample - loss: 0.0734 - acc: 0.9681 - val_loss: 0.4780 - val_acc: 0.8774
Epoch 7/10
596/596 [==============================] - 64s 107ms/sample - loss: 0.1492 - acc: 0.9446 - val_loss: 0.3067 - val_acc: 0.

# Custom CNN Model with Image Manipulations
## *This a stretch goal, and it's relatively difficult*

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [0]:
# State Code for Image Manipulation Here

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language